In [1]:
import pandas as pd
import numpy  as np

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
print(df.head(3))

                                              review  sentiment
0  In 1974, the teenager Martha Moxley (Maggie Gr...          1
1  OK... so... I really like Kris Kristofferson a...          0
2  ***SPOILER*** Do not read this, if you think a...          0


In [3]:

count = CountVectorizer()
docs  = np.array(['The sun is shining',
                   'The weather is sweet',
                  'The sun is shining, the weather is sweet,'
                  ' and one and one is two'
                ])

bag  = count.fit_transform(docs)

In [4]:
# fit_transorm first maps unique words to indices
print(count.vocabulary_) 

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [5]:
#Then, each string is converted into and array of number, index = the key of  word used in the sentencce and value = number of times
# that word was used in the string. These values are called raw term freq.
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [6]:
#term-freuquncy inverse documentfreuquency(tf-idf) is sed to downweight the freuquently occuring words 
#tf-idf = tf(t,d) * idf(t,d)

from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True
                        )
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [7]:
#Remove unwanted data like html tags and punctuations
print(df.loc[0,'review'][-50:])
import re
def preprocess(text):
    text = re.sub('<[^>]*>','',text)
    emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub(r'[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

print(preprocess(df.loc[0,'review'][-50:]))
print(preprocess("</a>This :) is :( a test :-)!"))

is seven.<br /><br />Title (Brazil): Not Available
is seven title brazil not available
this is a test :) :( :)


In [8]:
#Apply preprocessing to all data
df['review'] = df['review'].apply(preprocess)

In [9]:
print(df.loc[0,'review'][-50:])

zation my vote is seven title brazil not available


In [10]:
#Porter Stemming 
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

def tokenizer(text):
    return text.split()
print(tokenizer_porter('runner like running and thus they run'))


['runner', 'like', 'run', 'and', 'thu', 'they', 'run']


In [11]:
#Remove stop words or words which have provide no additional info like is, has etc
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop=stopwords.words('english')

[w for w in tokenizer_porter('a runner likes'
                             ' running and runs a lot') if w not in stop]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shubhsha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['runner', 'like', 'run', 'run', 'lot']

In [12]:
#Train logistic regression model for document classification
#Using Logistic regression as here we just need to classify review in 2 categories = positive and negative

X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [13]:
#Use GridSearchCV to find optimal parameters for logistic regression model using 5fold stratified cross-validation
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None
                       )

small_param_grid = [
    {
        'vect__ngram_range': [(1,1)],
        'vect__stop_words': [None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l2'],
        'clf__C': [1.0,10.0]
    },
    {
        'vect__ngram_range': [(1,1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer],
        'vect__use_idf': [False],
        'vect__norm': [None],
        'clf__penalty': ['l2'],
        'clf__C': [1.0,10.0]
    }
]

lr_tfidf = Pipeline([
                        ('vect', tfidf),
                        ('clf', LogisticRegression(solver='liblinear'))
                   ])

gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid, scoring='accuracy', cv=5, verbose=2, n_jobs=1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/home/shubhsha/projects/MachineLearning/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x77485112ce00>; total time=   4.5s


/home/shubhsha/projects/MachineLearning/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x77485112ce00>; total time=   5.0s


...

[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x77485112ce00>, vect__use_idf=False; total time=  17.5s


/home/shubhsha/projects/MachineLearning/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x77485112ce00>, vect__use_idf=False; total time=  16.8s


/home/shubhsha/projects/MachineLearning/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x77485112ce00>,
                                              <function tokenizer_porter at 0x77485377f740>...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x77485112ce00>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=2)

In [15]:
print(f'Best Parameter Set: {gs_lr_tfidf.best_params_}')

Best Parameter Set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x77485112ce00>}


In [16]:
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')

CV Accuracy: 0.897


In [17]:
clf = gs_lr_tfidf.best_estimator_

In [18]:
print(f'Test Accuracy: {clf.score(X_test, y_test):.3f}')

Test Accuracy: 0.899


In [25]:
print(clf.predict(['The best movie in history and the best ending in any entertainment business', "As for my rating, yes I'm one of those jerks who gives a movie a 1 just to lower it on the top 250. I think it's outrageous how highly it's ranked here. Even the folks who made it would think it's ridiculous to consider it one of the greatest films ever made.", "This film is nothing but one cliche after another. Having seen many of the 100's of prison films made from the early 30's to the 50's, I was able to pull almost every minute of Shawcrap from one of those films. While it is visually well made and acted, the story has every plot point of the standard prison film. They have the evil warden, innocent main character, friendly guilty guy, thugs and one good old prisoner. Don't waste your time on this one. Rent or buy some of the classic's of the prison genre."]))

[1 1 0]


In [21]:
print(X_test[0])

there s a part of me that would like to give this movie a high rating considering that it was made in 1953 this is a very courageous movie about transvestites tackling the issue fairly seriously and sympathetically and offering the viewer a lot of information on the subject and trying very hard not to stereotype the movie clearly makes the point that transvestites are not homosexuals and that aside from wearing women s clothing they lead a relatively normal life it deals with the pain of not being accepted in society the plot revolves around a police officer lyle talbot desperately trying to understand the issue because of the recent suicide of a transvestite so you have to give everyone involved with this movie credit for taking on such a controversial in the context of 1953 subject having said all that i m also sorry to say that this movie is absolutely dreadful in trying to portray glen glenda s edward d wood pain the movie falls into silly and at times surprisingly again given the 

In [27]:
import pickle
model_pkl_file = "sentiment_analysis_TFIDF_Logistic.pkl"
with open(model_pkl_file, 'wb') as file:
    pickle.dump(clf, file)